In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
# from src.metrics import precision_at_k, recall_at_k
from webinar5.utils import prefilter_items

from parameters.param_iterator import param_iterator

In [2]:
data = pd.read_csv('./retail_train.csv')

item_features = pd.read_csv('./product.csv')
user_features = pd.read_csv('./hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [3]:
item_features.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [4]:
user_features.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [5]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()

print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [6]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head()

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [7]:
data_test = data_test[data_test['item_id'].isin(data_train_filtered['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [9]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head()

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head()

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ
819255,1007.0,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - PORK,1 LB
819308,2879.0,MEAT,National,BEEF,CHOICE BEEF,
819400,584.0,GROCERY,National,DOG FOODS,DOG TREATS (SOFT TREATS),5.6 OZ


## Encoding features

In [11]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

## Model. Iterate over parameters

In [12]:
def calc_precision_for_parameters(param_grid, k):
    res_table = pd.DataFrame(columns=["parameters", f"train precision@{k}", f"test precision@{k}"])
    for param in param_iterator(param_grid, verbose=True):
        model = LightFM(random_state=42, **param)
        try:
            model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
                      sample_weight=coo_matrix(user_item_matrix),
                      user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                      item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                      epochs=15, 
                      num_threads=4,
                      verbose=False)
        except Exception as e:
            print("Error ocurred, skipping prameter")
            continue
    
        train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=k).mean()

        test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=k).mean()

        print(f"Result: train_precision = {train_precision}, test_precision = {test_precision}")
        result = pd.DataFrame([[param, train_precision, test_precision]], columns=["parameters", f"train precision@{k}", f"test precision@{k}"])
        res_table = res_table.append(result)
    return res_table


In [13]:
param_grid = {"no_components" : [ 16, 40, 64 ],
              "learning_rate" : [ 0.05, 0.5, 1],
              "item_alpha"    : [ 0.001, 0.01, 0.1 ],
              "user_alpha"    : [ 0.001, 0.01, 0.1 ],
              "max_sampled"   : [ 5, 10, 15 ],
              "loss"          : [ "warp", "bpr"] }
              
results = calc_precision_for_parameters(param_grid, 5)

Step 1 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.05, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.27785342931747437, test_precision = 0.002295253099873662
Step 2 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.05, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'bpr'}
Result: train_precision = 0.3527432978153229, test_precision = 0.0019822639878839254
Step 3 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.05, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'warp'}
Result: train_precision = 0.285142183303833, test_precision = 0.0017736046575009823
Step 4 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.05, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.35034042596817017, test_precision = 0.000938967103138566
Step 5 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.05, 'i

Result: train_precision = 0.17669205367565155, test_precision = 0.0016692748758941889
Step 37 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.05, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.09443332254886627, test_precision = 0.0010432967683300376
Step 38 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.05, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'bpr'}
Result: train_precision = 0.2041650116443634, test_precision = 0.002086593536660075
Step 39 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.05, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'warp'}
Result: train_precision = 0.22635163366794586, test_precision = 0.0012519562151283026
Step 40 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.05, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.20032040774822235, test_precision = 0.00187793

Result: train_precision = 0.1420104205608368, test_precision = 0.0012519562151283026
Step 73 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.5, 'item_alpha': 0.01, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.0793752521276474, test_precision = 0.003755868412554264
Step 74 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.5, 'item_alpha': 0.01, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'bpr'}
Result: train_precision = 0.14128956198692322, test_precision = 0.0017736046575009823
Step 75 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.5, 'item_alpha': 0.01, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'warp'}
Result: train_precision = 0.07561073452234268, test_precision = 0.0027125715278089046
Step 76 of 486
Parameters: {'no_components': 16, 'learning_rate': 0.5, 'item_alpha': 0.01, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.14048859477043152, test_precision = 0.000834637

Result: train_precision = 0.11317581683397293, test_precision = 0.0023995828814804554
Step 109 of 486
Parameters: {'no_components': 16, 'learning_rate': 1, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.13504205644130707, test_precision = 0.0023995826486498117
Step 110 of 486
Parameters: {'no_components': 16, 'learning_rate': 1, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'bpr'}
Result: train_precision = 0.13872648775577545, test_precision = 0.0008346374379470944
Step 111 of 486
Parameters: {'no_components': 16, 'learning_rate': 1, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'warp'}
Result: train_precision = 0.10292350500822067, test_precision = 0.003234220203012228
Step 112 of 486
Parameters: {'no_components': 16, 'learning_rate': 1, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.15010014176368713, test_precision = 0.0036515

Result: train_precision = 0.15394474565982819, test_precision = 0.0023995828814804554
Step 145 of 486
Parameters: {'no_components': 16, 'learning_rate': 1, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.15778936445713043, test_precision = 0.002086593536660075
Step 146 of 486
Parameters: {'no_components': 16, 'learning_rate': 1, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 5, 'loss': 'bpr'}
Result: train_precision = 0.15074089169502258, test_precision = 0.0018779343226924539
Step 147 of 486
Parameters: {'no_components': 16, 'learning_rate': 1, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'warp'}
Result: train_precision = 0.16331599652767181, test_precision = 0.003025560872629285
Step 148 of 486
Parameters: {'no_components': 16, 'learning_rate': 1, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.18646377325057983, test_precision = 0.0014606156619265

...

Result: train_precision = 0.29106929898262024, test_precision = 0.0016692748758941889
Step 328 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.05, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.2020024061203003, test_precision = 0.0015649452107027173
Step 329 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.05, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'warp'}
Result: train_precision = 0.2919503450393677, test_precision = 0.0021909233182668686
Step 330 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.05, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'bpr'}
Result: train_precision = 0.34873849153518677, test_precision = 0.0017736046575009823
Step 331 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.05, 'item_alpha': 0.001, 'user_alpha': 0.01, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.2905086576938629, test_precision =

Result: train_precision = 0.2375650852918625, test_precision = 0.0007303077727556229
Step 364 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.05, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.1704445481300354, test_precision = 0.002086593536660075
Step 365 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.05, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'warp'}
Result: train_precision = 0.18806569278240204, test_precision = 0.0016692748758941889
Step 366 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.05, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'bpr'}
Result: train_precision = 0.20328395068645477, test_precision = 0.0017736046575009823
Step 367 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.05, 'item_alpha': 0.1, 'user_alpha': 0.01, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.0918702483177185, test_precision = 0.000417

Result: train_precision = 0.06167401000857353, test_precision = 0.002086593536660075
Step 400 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.5, 'item_alpha': 0.01, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.150020033121109, test_precision = 0.0023995826486498117
Step 401 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.5, 'item_alpha': 0.01, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'warp'}
Result: train_precision = 0.08858630806207657, test_precision = 0.0015649452107027173
Step 402 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.5, 'item_alpha': 0.01, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'bpr'}
Result: train_precision = 0.1465758979320526, test_precision = 0.002295253099873662
Step 403 of 486
Parameters: {'no_components': 64, 'learning_rate': 0.5, 'item_alpha': 0.01, 'user_alpha': 0.01, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.07969563454389572, test_precision = 0.0027125

Result: train_precision = 0.07705245912075043, test_precision = 0.002921231323853135
Step 436 of 486
Parameters: {'no_components': 64, 'learning_rate': 1, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.14409291744232178, test_precision = 0.0012519562151283026
Step 437 of 486
Parameters: {'no_components': 64, 'learning_rate': 1, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'warp'}
Result: train_precision = 0.10997197031974792, test_precision = 0.0029212310910224915
Step 438 of 486
Parameters: {'no_components': 64, 'learning_rate': 1, 'item_alpha': 0.001, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'bpr'}
Result: train_precision = 0.14305166900157928, test_precision = 0.0008346374379470944
Step 439 of 486
Parameters: {'no_components': 64, 'learning_rate': 1, 'item_alpha': 0.001, 'user_alpha': 0.01, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.09098919481039047, test_precision = 0.0007303

Result: train_precision = 0.15194234251976013, test_precision = 0.0021909233182668686
Step 472 of 486
Parameters: {'no_components': 64, 'learning_rate': 1, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 10, 'loss': 'bpr'}
Result: train_precision = 0.1892671287059784, test_precision = 0.0010432967683300376
Step 473 of 486
Parameters: {'no_components': 64, 'learning_rate': 1, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'warp'}
Result: train_precision = 0.16123349964618683, test_precision = 0.0019822639878839254
Step 474 of 486
Parameters: {'no_components': 64, 'learning_rate': 1, 'item_alpha': 0.1, 'user_alpha': 0.001, 'max_sampled': 15, 'loss': 'bpr'}
Error ocurred, skipping prameter
Step 475 of 486
Parameters: {'no_components': 64, 'learning_rate': 1, 'item_alpha': 0.1, 'user_alpha': 0.01, 'max_sampled': 5, 'loss': 'warp'}
Result: train_precision = 0.152102530002594, test_precision = 0.0019822639878839254
Step 476 of 486
Parameters: {'no_components': 64, 

In [16]:
results.sort_values(by="test precision@5", inplace=True, ascending=False)
results.head()

,parameters,train precision@5,test precision@5
0,"{'no_components': 64, 'learning_rate': 1, 'ite...",0.071766,0.011894
0,"{'no_components': 64, 'learning_rate': 0.5, 'i...",0.107249,0.008033
0,"{'no_components': 40, 'learning_rate': 0.05, '...",0.187425,0.007720
0,"{'no_components': 40, 'learning_rate': 0.5, 'i...",0.155066,0.006468
0,"{'no_components': 64, 'learning_rate': 0.5, 'i...",0.154746,0.006155


In [18]:
print(f"Best parameters: {results.iloc[0]['parameters']}")

Best parameters: {'no_components': 64, 'learning_rate': 1, 'item_alpha': 0.001, 'user_alpha': 0.01, 'max_sampled': 10, 'loss': 'warp'}
